Dash 

Instalar o Dash

Estrutura Básica

plotly-Gráficos
flask - aplicação

Primeiro DashBoard

Como Funciona

Layout
    HTML
    Dash Components (Core Components)

CallBacks

In [1]:

from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import dash_auth

USUARIOS = {
    "Lira": "123456",
    "Hashtag": "98765",
}


app = Dash(__name__) # criando o seu aplicativo Dash
auth = dash_auth.BasicAuth(app, USUARIOS)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.read_excel("Vendas.xlsx",engine='openpyxl')
# df = tabela = dataframe

# plotly
fig = px.bar(df, x="Produto", y="Quantidade", color="ID Loja", barmode="group")
fig2 = px.scatter(df, x="Quantidade", y="Valor Final", color="Produto", size="Valor Unitário", size_max=60)


# lista_marcas = ["Treinamentos", "Programação", "Todas"]
lista_marcas = list(df["Marca"].unique())
lista_marcas.append("Todas")

lista_paises = list(df["País"].unique())
lista_paises.append("Todos")

# layout
app.layout = html.Div(children=[
    html.H1(children='Meu Dashboard'),

    html.Div(children='''
        Dashboard de Vendas em Python
    '''),
    
    html.H3(children="Vendas de cada Produto por Loja", id="subtitulo"),
    
    dcc.RadioItems(lista_marcas, value="Todas", id='selecao_marcas'),
    html.Div(children=[
        dcc.Dropdown(lista_paises, value="Todos", id='selecao_pais'),
    ], style={"width": "50%", "margin": "auto"}),
    
    dcc.Graph(id='vendas_por_loja',figure=fig),
    dcc.Graph(id='distribuicao_vendas', figure=fig2),
    
    
], style={"text-align": "center"})

# callbacks -> dar funcionalidade pro nosso dashboard (conecta os botões com os gráficos)
@app.callback(
    Output('subtitulo', 'children'), # eu quero modificar (eu quero que o botão do input modifique)
    Output('vendas_por_loja', 'figure'),
    Output('distribuicao_vendas', 'figure'),
    Input('selecao_marcas', 'value'), # quem está modificando/de onde eu quero pegar a informacao/que tá fazendo um filtro
    Input('selecao_pais', 'value'),
)
def selecionar_marca(marca, pais):
    if marca == "Todas" and pais == "Todos":
        texto = "Vendas de cada Produto por Loja"
        fig = px.bar(df, x="Produto", y="Quantidade", color="ID Loja", barmode="group")
        fig2 = px.scatter(df, x="Quantidade", y="Valor Final", color="Produto", size="Valor Unitário", size_max=60)
    else:
        df_filtrada = df
        if marca != "Todas":
            #filtrar de acordo com a marca
            df_filtrada = df_filtrada.loc[df_filtrada['Marca']==marca, :]
        if pais != "Todos":
            # filtrar de acordo com o pais
            df_filtrada = df_filtrada.loc[df_filtrada["País"]==pais, :]
        
        texto = f"Vendas de cada Produto por Loja da Marca {marca} e do País {pais}"
        fig = px.bar(df_filtrada, x="Produto", y="Quantidade", color="ID Loja", barmode="group")
        fig2 = px.scatter(df_filtrada, x="Quantidade", y="Valor Final", color="Produto", size="Valor Unitário", size_max=60)
    return texto, fig, fig2


# colocando o seu site (seu dashboard) no ar
if __name__ == '__main__':
    app.run_server(debug=False)


Exception: 